# Task 2: Coordinate Transformations

In this notebook we will use what we have learned about marker detection in the last notebook

## Imports

In [2]:
# imports
import cv2
from matplotlib import pyplot as plt
import numpy as np
import yaml
import io

## Setup

We again load the camera parameters. This time not only need them for preprocessing our image, but we will also use them when estimating where the markers are relative to the camera.

In [3]:
# load camera parameters
with open("camera_intrinsics.yml", 'r') as stream:
    camera_intrinsics = yaml.safe_load(stream)
camera_matrix = np.array(camera_intrinsics["camera_matrix"]["data"]).reshape((3,3))
distortion_coefficients = np.array(camera_intrinsics["distortion_coefficients"]["data"])

The other component for estimating the markers' poses and postions we need to know beforehand, is the size of our markers.

In [4]:
# set marker size
marker_size = 0.05

## Detect Aruco Markers

**Task:** Use the *cv2.aruco.DICT_ARUCO_ORIGINAL* dictionary for detecting the markers in the given image. 

**Note:** In this specific case, you don't have to undistort the image after loading it, because the function that handles the marker pose estimation (*solvePnP*, which we will use in the next task) will do that internally. So you just have to detect the markers in the undistorted image, image distortions will be accounted for post detection.

**Note:** The list that contains the marker ids should be named *ids*, in order for the provided code to run without changes.

In [ ]:
# load image
img = cv2.imread("./2024-09-12_17-29-34/img_282.jpg")

### Your code here ###

# setup detector


# detect markers in the image


# draw markers onto the image
img_with_markers = None
###


plt.imshow(img_with_markers)

## Pose Estimation

**Task:** Use OpenCV's *solvePnP* function to estimate the poses of the markers detected in the previous task. 

**Note:** The function should get the camera matrix and distortion parameters as arguments, because it internally handles the *undistortion* step that we did separately in the last notebook.

In [6]:
# estimate marker poses using the detected marker corners from the image, the camera parameters, and the (known) size of the marker 
marker_rvecs = []
marker_tvecs = []

# marker corners
marker_points = np.array([[-marker_size / 2, marker_size / 2, 0],
                          [marker_size / 2, marker_size / 2, 0],
                          [marker_size / 2, -marker_size / 2, 0],
                          [-marker_size / 2, -marker_size / 2, 0]])

### Your code here ###


###

**Task:** Use OpenCV's *drawFrameAxes* function to visualise the estimated marker poses.

In [ ]:
# visualise the estimated marker poses in the image
img_axis = img.copy()
### Your code here ###

###

plt.imshow(img_axis)

## Projecting Aruco Marker Positions to Robot and World Frame

In [8]:
def rotation_matrix(x_rotation, y_rotation, z_rotation):
    alpha = z_rotation
    beta = y_rotation
    gamma = x_rotation

    # https://en.wikipedia.org/wiki/Rotation_matrix
    # rotates around z, then y, then x

    return np.array([
        [np.cos(alpha)*np.cos(beta), np.cos(alpha)*np.sin(beta)*np.sin(gamma)-np.sin(alpha)*np.cos(gamma), np.cos(alpha)*np.sin(beta)*np.cos(gamma)+np.sin(alpha)*np.sin(gamma)],
        [np.sin(alpha)*np.cos(beta), np.sin(alpha)*np.sin(beta)*np.sin(gamma)+np.cos(alpha)*np.cos(gamma), np.sin(alpha)*np.sin(beta)*np.cos(gamma)-np.cos(alpha)*np.sin(gamma)],
        [-np.sin(beta), np.cos(beta)*np.sin(gamma), np.cos(beta)*np.cos(gamma)]
    ])

In [9]:
def vectors_to_transformation_matrix(rotation, translation):
    ### Your code here ###

    ###
    return T

In [10]:
def transformation_matrix_to_vectors(t):
    ### Your code here ###

    ###
    return np.array([x_rotation, y_rotation, z_rotation]), translation

### Projecting to robot frame

In [11]:
# offset from camera to robot frame
x_offset_camera = 0.03
y_offset_camera = 0
z_offset_camera = 0.28

offset_camera_robot = np.array([x_offset_camera,y_offset_camera,z_offset_camera]) 

In [12]:
# angle from camera to robot frame
x_angle_camera = -140 #degrees
y_angle_camera = 0
z_angle_camera = -90 #degrees

rotation_camera_robot = np.array([np.radians(x_angle_camera), np.radians(y_angle_camera), np.radians(z_angle_camera)])

**Task:** Create transform matrix from camera frame to robot frame.

In [13]:
### Your code here ###
T_camera_robot = None
###

**Task:** Create transform matrices for all markers, project them into the robot coordinate frame, and transform the resulting matrices back into rotation and translation vectors. Also print the id and world position for each marker.

In [ ]:
# create transform matrices for all markers, project them into the robot coordinate frame
# transform them to rotation and translation vectors
marker_rvecs_robot = []
marker_tvecs_robot = []
### Your code here ###

###

Now, let us visualise the results of our pose estimation. We start with a visualisation of where the detected markers are, realitve to our robot, on the vertical plane. 

In [ ]:
for i, c in enumerate(marker_tvecs_robot):
    id = ids[i]
    plt.scatter([c[1]], [c[2]], color="black", marker="s")
    plt.text(c[1], c[2], str(id))
    plt.ylabel("z")
    plt.xlabel("y")
    plt.xlim([-0.3, 0.3])
    plt.ylim([-0.5, 0.5])
    plt.axhline(0,-1,1, color="lightgray", ls="--")
    plt.text(0.25, 0.01, "floor", color="lightgray")

plt.gca().invert_xaxis()
plt.title("vertical plane")

We can see that, although all markers in the image are obviously placed directly on the floor, our estimated positions don't line uo perfectly with it. Some markers are estimated to "float" a bit above the floor, some are estimated to be slightly below it. This kind of imperfect measurement is something we must always expect when working with imperfect data. In our example, our camera calibration might not be completely optimal our images might be noisy, etc. Overall, however, these results aren't too bad and definitely still usable for our purposes. 

Next, we visualise where the markers are on the horizontal plane (i.e. the floor), relative to our robot. Because the robot is the reference point, it will always be at position (0,0).

In [ ]:
for i, c in enumerate(marker_tvecs_robot):
    id = ids[i]
    plt.scatter([c[1]], [c[0]], color="black", marker="s")
    plt.text(c[1], c[0], str(id))
    plt.ylabel("x [m]")
    plt.xlabel("y [m]")
plt.xlim(-1, 1)
plt.ylim(-1, 1)
plt.scatter([0], [0], color="red", marker="s")
plt.text(0, 0, "Robot")
plt.gca().invert_xaxis()
plt.title("horizontal plane")

### Projecting to world frame

Now that we know, where the markers are relative to our robot, we want to determine, where they are in our world coordinate frame. We will later need this to, for example, implement the landmark detection and localisation, as well as the SLAM algorithm for our robot.  
Let's assume that our robot started at the origin of the world coordinate frame. Now it has moved 40cm forwards and 25cm to the left, with a current heading o 30° to the right.


In [ ]:
# offset from robot to world frame
x_offset_world = 0.4
y_offset_world = 0.25
z_offset_world = 0

# angle from robot to world frame
x_angle_world = 0 #degrees
y_angle_world = 0
z_angle_world = -30

offset_robot_world = np.array([x_offset_world,y_offset_world,z_offset_world]) 
rotation_robot_world = np.array([np.radians(x_angle_world), np.radians(y_angle_world), np.radians(z_angle_world)])

**Task:** Create transform matrix from robot to world frame.

In [20]:
### Your code here ###
T_robot_world = None
###

**Task:** Create transform matrices for all markers in the robot coordinate frame, project them into the world coordinate frame, and transform them to rotation and translation vectors. Also print the id and world position for each marker.

In [ ]:
marker_rvecs_world = []
marker_tvecs_world = []
### Your code here ###

###

Let's now again visualise our results. First, we plot where our markers are relative to the floor.  

In [ ]:
for i, c in enumerate(marker_tvecs_world):
    id = ids[i]
    # we mirror the y coordinate, because in the robot coordinate system y < 0 indicates a position to the right of the 
    # robot, but in the pyplot visualisation values < 0 are displayed on the left. The x ticks are adjusted accordingly.
    plt.scatter([c[1]], [c[2]], color="black", marker="s")
    plt.text(c[1], c[2], str(id))
    plt.ylabel("z [m]")
    plt.xlabel("y [m]")
    plt.xlim([0.2, 0.8])
    plt.ylim([-0.5, 0.5])
    plt.axhline(0,-1,1, color="lightgray", ls="--")
    plt.text(0.22, 0.01, "floor", color="lightgray")
    plt.xticks([0.3,0.4,0.5,0.6,0.7], [-0.3,-0.4,-0.5,-0.6,-0.7])
plt.title("vertical plane")

The measurement error from the previous pose estimation step obviously still exists, but overall our measurement is still good enough to use it, for example for navigation.

Finally, we again visualise the positions of the markers in the horizontal plane. This time we also visualise the position of our robot. This gives us essentially a section of our world map.

In [ ]:
for i, c in enumerate(marker_tvecs_world):
    id = ids[i]
    plt.scatter([c[1]], [c[0]], color="black", marker="s")
    plt.text(c[1], c[0], str(id))
    plt.ylabel("x [m]")
    plt.xlabel("y [m]")
plt.xlim(-1, 1.5)
plt.ylim(-1, 1.5)
plt.gca().invert_xaxis()
plt.scatter([y_offset_world], [x_offset_world], color="red", marker="s")
plt.text(y_offset_world, x_offset_world, "Robot")
plt.title("horizontal plane")